In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Spark ML ex") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

sc = spark.sparkContext

22/11/03 16:12:57 WARN Utils: Your hostname, orange resolves to a loopback address: 127.0.1.1; using 166.104.246.51 instead (on interface enp15s0)
22/11/03 16:12:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 16:12:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/03 16:12:59 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [9]:
import numpy as np
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt
from pyspark.ml.linalg import Vectors

x, y = np.random.multivariate_normal([0, 0], [[6, -2], [-2, 1]], 100).T
dataset = spark.createDataFrame([[Vectors.dense([x, y])] for x, y in zip(x, y)], ['point'])

print(x.shape, y.shape)
print(dataset.show(5, truncate=False))
# print(dataset.count())

(100,) (100,)
+-----------------------------------------+
|point                                    |
+-----------------------------------------+
|[-0.2459447501136685,-0.5144881519861378]|
|[4.156844727038653,-2.4387409463314405]  |
|[2.1621355273399776,0.3229967399991434]  |
|[1.9392233464120734,-0.4069024580039728] |
|[1.0824445524368689,-1.2616416748204813] |
+-----------------------------------------+
only showing top 5 rows

None


In [11]:
pca = PCA(k=1, inputCol="point", outputCol="features")
model = pca.fit(dataset)
reduced = model.transform(dataset)
reduced.show(5, truncate=False)

+-----------------------------------------+---------------------+
|point                                    |features             |
+-----------------------------------------+---------------------+
|[-0.2459447501136685,-0.5144881519861378]|[0.04236141642319363]|
|[4.156844727038653,-2.4387409463314405]  |[-4.758630199928948] |
|[2.1621355273399776,0.3229967399991434]  |[-1.8973148679115435]|
|[1.9392233464120734,-0.4069024580039728] |[-1.9546254055153696]|
|[1.0824445524368689,-1.2616416748204813] |[-1.466652105333682] |
+-----------------------------------------+---------------------+
only showing top 5 rows



In [14]:
model.pc

DenseMatrix(2, 1, [-0.9318, 0.3631], 0)

In [ ]:
plt.plot(dataset.select('point'))

In [30]:
dataset.select('point').collect()[0].point

DenseVector([-0.2459, -0.5145])